In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
df=pd.read_csv('PenDigits_withoutdupl_norm_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,outlier
0,48.0,100.0,20.0,80.0,0.0,51.0,55.0,43.0,100.0,56.0,90.0,62.0,86.0,30.0,73.0,0.0,1
1,38.0,100.0,4.0,85.0,0.0,53.0,58.0,46.0,100.0,52.0,88.0,64.0,78.0,32.0,73.0,0.0,1
2,53.0,100.0,25.0,70.0,0.0,39.0,50.0,34.0,100.0,42.0,85.0,64.0,77.0,32.0,80.0,0.0,1
3,12.0,100.0,0.0,71.0,25.0,43.0,86.0,44.0,100.0,75.0,90.0,52.0,90.0,20.0,90.0,0.0,1
4,26.0,100.0,0.0,83.0,23.0,51.0,83.0,42.0,100.0,55.0,79.0,84.0,56.0,42.0,37.0,0.0,1


In [3]:
df.shape

(9868, 17)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Pendigits.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,16]
X = df[:,0:16]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/kader/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
           # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 11:25:16,829][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 16)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 16)],y_test.shape=(3257,)
[ 2018-05-15 11:25:16,833][cascade_classifier.fit_transform] group_dims=[16]
[ 2018-05-15 11:25:16,834][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 11:25:16,836][cascade_classifier.fit_transform] group_ends=[16]
[ 2018-05-15 11:25:16,837][cascade_classifier.fit_transform] X_train.shape=(6611, 16),X_test.shape=(3257, 16)
[ 2018-05-15 11:25:16,840][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 16), X_cur_test.shape=(3257, 16)
[ 2018-05-15 11:25:18,354][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:25:19,206][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:25:19,934][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 11:25:25,088][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-05-15 11:25:25,090][cascade_classifier.transform] group_dims=[16]
[ 2018-05-15 11:25:25,092][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-05-15 11:25:25,093][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


Test Accuracy of ExtraTrees = 99.877188 %


In [10]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[3249    0]
 [   4    4]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.50      0.67         8

avg / total       1.00      1.00      1.00      3257



In [11]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 11:25:26,575][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-05-15 11:25:26,576][cascade_classifier.transform] group_dims=[16]
[ 2018-05-15 11:25:26,578][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-05-15 11:25:26,580][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


In [12]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 11:25:27,752][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-05-15 11:25:27,755][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-05-15 11:25:27,756][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 11:25:27,758][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-05-15 11:25:27,765][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-05-15 11:25:27,881][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-05-15 11:25:28,510][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:25:29,103][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:25:29,818][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [13]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 11:25:35,143][cascade_classifier.transform] X_groups_test.shape=[(3257, 18)]
[ 2018-05-15 11:25:35,145][cascade_classifier.transform] group_dims=[18]
[ 2018-05-15 11:25:35,146][cascade_classifier.transform] X_test.shape=(3257, 18)
[ 2018-05-15 11:25:35,149][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 18)


Test Accuracy of ExtraTrees = 99.938594 %


In [14]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[3249    0]
 [   2    6]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.75      0.86         8

avg / total       1.00      1.00      1.00      3257



In [15]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [16]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 11:25:44,570][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-05-15 11:25:44,574][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-05-15 11:25:44,576][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 11:25:44,577][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-05-15 11:25:44,579][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-05-15 11:25:44,581][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-05-15 11:25:45,065][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:25:45,657][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:25:46,364][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')
[[3249    0]
 [   1    7]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.88      0.93         8

avg / total       1.00      1.00      1.00      3257



[ 2018-05-15 11:25:53,471][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:25:54,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:25:55,042][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-05-15 11:25:55,771][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-05-15 11:25:56,575][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-05-15 11:25:57,372][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-05-15 11:25:58,760][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-05-15 11:25:59,508][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 1, 'fois')
[[3249    0]
 [   1    7]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.88      0.93         8

avg / total       1.00      1.00      1.00      3257



[ 2018-05-15 11:26:02,975][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:26:03,569][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:26:04,305][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-05-15 11:26:05,021][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-05-15 11:26:05,735][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-05-15 11:26:06,433][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-05-15 11:26:07,179][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-05-15 11:26:07,953][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

...

('Test Accuracy of ExtraTrees = 99.969297 %', 17, 'fois')
[[3249    0]
 [   1    7]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.88      0.93         8

avg / total       1.00      1.00      1.00      3257



[ 2018-05-15 11:28:17,902][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:28:18,498][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:28:19,241][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-05-15 11:28:19,843][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-05-15 11:28:20,444][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-05-15 11:28:21,026][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-05-15 11:28:21,609][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-05-15 11:28:22,349][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 18, 'fois')
[[3249    0]
 [   1    7]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.88      0.93         8

avg / total       1.00      1.00      1.00      3257



[ 2018-05-15 11:28:25,616][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-05-15 11:28:26,205][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-05-15 11:28:26,811][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-05-15 11:28:27,547][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-05-15 11:28:28,157][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-05-15 11:28:28,734][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-05-15 11:28:29,314][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-05-15 11:28:29,902][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

('Test Accuracy of ExtraTrees = 99.969297 %', 19, 'fois')
[[3249    0]
 [   1    7]]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      3249
        1.0       1.00      0.88      0.93         8

avg / total       1.00      1.00      1.00      3257



# ('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')